In [2]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

In [3]:
os.getcwd()

'/home/nick/classwork_21_22/COMP755/COMP755FinalProj/Processing'

In [5]:
celeba256inpainted_f = sorted(glob('../../celeba_256_inpainted_gatedconv/*'))
print(len(celeba256inpainted_f))
print(celeba256inpainted_f[:5])


15028
['../../celeba_256_inpainted_gatedconv/000001.jpg_256_masked.png_inpainted.png', '../../celeba_256_inpainted_gatedconv/000002.jpg_256_masked.png_inpainted.png', '../../celeba_256_inpainted_gatedconv/000003.jpg_256_masked.png_inpainted.png', '../../celeba_256_inpainted_gatedconv/000004.jpg_256_masked.png_inpainted.png', '../../celeba_256_inpainted_gatedconv/000006.jpg_256_masked.png_inpainted.png']


In [7]:
import pickle

with open('../../objects_jsons/celeba_test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('../../objects_jsons/celeba_train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)

test_set = list(test_set)
train_set = list(train_set)

print(test_set[:5])
print(train_set[:5])

test_set_f = [x[0] for x in test_set]
test_set_labels = [x[1] for x in test_set]

train_set_f = [x[0] for x in train_set]
train_set_labels = [x[1] for x in train_set]


[('096517.jpg', 'Danielle_Evans'), ('047646.jpg', 'Alice_Greczyn'), ('080864.jpg', 'Dev_Patel'), ('066619.jpg', 'Adrian_Pasdar'), ('150758.jpg', 'Jorma_Taccone')]
[('000885.jpg', 'Bryan_Ferry'), ('142160.jpg', 'Jarvis_Cocker'), ('156848.jpg', 'Maky_Soler'), ('147496.jpg', 'Katarzyna_Glinka'), ('024698.jpg', 'Carly_Pope')]


In [34]:
zoom_factor = 224/256

for i in range(len(celeba256inpainted_f)):
    # print(i / len(celeba256inpainted_f))
    inpainted_f = celeba256inpainted_f[i]
    original_f = os.path.basename(inpainted_f)[:10]
    # print(original_f)

    train_ind = None
    test_ind = None
    label = None
    try:
        train_ind = train_set_f.index(original_f)
        label = train_set_labels[train_ind]
    except ValueError:
        pass

    try:
        test_ind = test_set_f.index(original_f)
        label = test_set_labels[test_ind]
    except ValueError:
        pass
        
    savename = original_f + "_inpainted_224.png"
    savedir = None
    if train_ind is None:
        savedir = '../../dataset/val/' + label + '/'
    elif test_ind is None:
        savedir = '../../dataset/train/' + label + '/'
    else:
        print("error: img not in test set or train set")
        print("img: " + original_f)
        continue

    if not os.path.isdir(savedir):
        os.mkdir(savedir)

    if os.path.isfile(os.path.join(savedir, savename)):
        continue

    img_256 = plt.imread(inpainted_f)
    # print(img_256.shape)
    try:
        assert(img_256.shape == (256, 256, 3))
    except AssertionError:
        # print(original_f)
        # print(label)
        # print(img_256.shape)

        tmp = np.zeros((256,256,3))
        buf = 256 - img_256.shape[0]
        try:
            assert(buf == 8)
        except AssertionError:
            print(original_f)
            print(img_256.shape)
            continue
        tmp[4: -4,:,:] = img_256
        tmp[:4,:,:] = img_256[::-1,:,:][-4:,:,:]
        tmp[-4:,:,:] = img_256[::-1,:,:][:4,:,:]
        img_256 = tmp

    assert(img_256.shape == (256,256,3))

    img_224 = zoom(img_256, (zoom_factor, zoom_factor, 1))
    assert(img_224.shape[0] == 224)
    assert(img_224.shape[1] == 224)

    img_224 += np.abs(np.min([np.min(img_224), 0.]))
    img_224 /= np.max([np.max(img_224), 1.])
    # print(np.max(img_224), np.min(img_224))
    plt.imsave(os.path.join(savedir,savename), img_224)


001786.jpg
(208, 256, 3)
099801.jpg
(232, 256, 3)


In [25]:
a = np.arange(25).reshape((5,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [29]:
a[1:3:-1,:]

array([], shape=(0, 5), dtype=int64)